In [2]:
import pandas as pd, numpy as np, os, math

In [3]:
outstanding_games = pd.read_csv("2020GamesLeft.csv")

In [4]:
allTheStats = pd.read_csv("../perfectlyCleaned_Data.csv")

In [7]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [8]:
outstanding_games.columns

Index(['Year', 'Week2', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'GAMEID', 'BINARY_WIN', 'Home_Team'],
      dtype='object')

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [10]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [11]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [15]:
model_1 = sklearn.linear_model.LinearRegression()

In [13]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [17]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
#R_sq Score
model_1.score(X_train, y_train)

0.11160052247107721

In [21]:
model_1.score(X_test, y_test)

0.12100326484896597

In [22]:
model_1.intercept_

array([22.64165239])

In [23]:
model_1.coef_

array([[0.64681882]])

In [25]:
y_pred = model_1.predict(X_test)

In [26]:
sq_resid = (y_pred - y_test)**2

In [27]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [30]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_1, params, verbose=3)

In [31]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.121, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.045, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.105, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.131, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [33]:
grid.best_score_

0.10659290608447188

In [34]:
ys = grid.predict(X_test)

In [35]:
resids = (ys - y_test)**2

In [36]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [37]:
from sklearn.metrics import r2_score

In [39]:
r2_score(y_test, ys)

0.12100326484896595

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [40]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [43]:
allTheStats2 = allTheStats.copy(deep=True)

In [44]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [50]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [78]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [80]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [74]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
dtypes: float64(7)
memory usage: 181.4 KB


In [79]:
model_values.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', 'Power Index DEF_Opp']]

,FINAL_TEAM_NAME,Power Index DEF,Power Index DEF_Opp
0,Detroit Lions,-2.4,-2.7
1,Chicago Bears,-2.7,-2.4
2,Chicago Bears,-2.7,-2.4
3,Detroit Lions,-2.4,-2.7
4,Washington Redskins,-2.2,-2.7
...,...,...,...
2897,Kansas City Chiefs,1.0,-0.2
2898,Oakland Raiders,-3.4,1.3
2899,New England Patriots,1.3,-3.4
2900,Buffalo Bills,-0.2,-3.4


In [87]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [88]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [91]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [92]:
#R_sq Score
model_2.score(X_train, y_train)

0.2117845417196058

In [93]:
model_2.score(X_test, y_test)

0.19001364187854686

In [94]:
model_2.intercept_

array([24.06222075])

In [95]:
model_2.coef_

array([[ 0.04133081,  0.67166743, -0.95707838,  0.47373876,  0.00707634,
        -0.09042084]])

In [96]:
y_pred = model_2.predict(X_test)

In [97]:
sq_resid = (y_pred - y_test)**2

In [98]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [99]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [100]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.245, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.191, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.228, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.209, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.153, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.245, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [101]:
grid.best_score_

0.20522667173526582

In [102]:
ys = grid.predict(X_test)

In [103]:
resids = (ys - y_test)**2

In [104]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [105]:
from sklearn.metrics import r2_score

In [106]:
r2_score(y_test, ys)

0.19001364187854686

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [107]:
allTheStats2.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [108]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [109]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [112]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [110]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [111]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [113]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [114]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [120]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [121]:
#R_sq Score
model_3.score(X_train, y_train)

0.4887859670361537

In [122]:
model_3.score(X_test, y_test)

0.5273383363155932

In [123]:
model_3.intercept_

array([4.07569457])

In [124]:
model_3.coef_

array([[ 0.08343974,  0.11082988, -0.34514904,  0.36145503,  0.01845262,
        -0.08814872,  0.0644413 , -1.81601954]])

In [125]:
y_pred = model_3.predict(X_test)

In [126]:
sq_resid = (y_pred - y_test)**2

In [127]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [128]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [129]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.401, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.495, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.477, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.508, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.513, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.401, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [130]:
grid.best_score_

0.4785656440179504

In [131]:
ys = grid.predict(X_test)

In [132]:
resids = (ys - y_test)**2

In [133]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [134]:
from sklearn.metrics import r2_score

In [135]:
r2_score(y_test, ys)

0.5273383363155932

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [136]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [137]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [138]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [139]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
rf = RandomForestClassifier(n_estimators=4000, criterion="entropy", max_depth=5)

In [141]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [143]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [144]:
#R_sq Score
rf.score(X_train, y_train)

0.25505514705882354

In [145]:
rf.score(X_test, y_test)

0.08677685950413223

In [146]:
rf.classes_

array([ 0.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 51., 52., 54., 55., 57.,
       59.])

In [152]:
y_pred = rf.predict(X_test)

In [153]:
sq_resid = (y_pred - y_test)**2

In [154]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.420603480333007

In [155]:
## STOP POINT FOR RF ##

In [162]:
from sklearn.model_selection import RandomizedSearchCV

In [163]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [164]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  29.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.2s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  31.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


...

[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.074, total=  24.5s
[CV] n_estimators=8000, max_depth=10, criterion=gini .................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.092, total=  24.3s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  6.5min finished
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [165]:
grid.best_score_

0.09328271643994517

In [166]:
ys = grid.predict(X_test)

In [167]:
resids = (ys - y_test)**2

In [168]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.918588590639681

In [169]:
from sklearn.metrics import r2_score

In [170]:
r2_score(y_test, ys)

0.214525684460524

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [171]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [173]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [174]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2902 non-null   float64
 1   Power Index FPI                2902 non-null   float64
 2   Power Index OFF                2902 non-null   float64
 3   Power Index DEF_Opp            2902 non-null   float64
 4   Power Index ST                 2902 non-null   float64
 5   Ranks SOS                      2902 non-null   float64
 6   Ranks AVGWP                    2902 non-null   float64
 7   TEAM_YARDS                     2902 non-null   float64
 8   TEAM_TURNOVERS                 2902 non-null   float64
 9   EPA/play_RDBMS_Winner          2902 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2902 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2902 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2902 non-null   f

In [175]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [176]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [178]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [179]:
#R_sq Score
model_5.score(X_train, y_train)

0.6302700899587667

In [180]:
model_5.score(X_test, y_test)

0.6277712268609107

In [181]:
model_5.intercept_

array([19.35315188])

In [182]:
model_5.coef_

array([[ 1.30057265e-01, -8.14878619e-02, -2.34768717e-02,
         8.55711147e-01,  2.11939999e-02, -5.30486944e-02,
         2.24907272e-02,  5.33776096e-01,  2.75554251e+01,
        -3.30534744e+01,  4.57205456e+00,  1.49555454e+01,
         2.51527195e+00,  7.17865874e+00]])

In [204]:
model_values

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_YARDS,TEAM_TURNOVERS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
0,24.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,349.0,0.0,0.180,0.468,0.257,0.500,-0.041,0.375
1,20.0,-2.6,0.1,-2.4,0.0,12.0,27.0,345.0,3.0,-0.008,0.442,-0.009,0.483,-0.007,0.391
2,34.0,-2.6,0.1,-2.4,0.0,12.0,27.0,444.0,1.0,0.053,0.436,0.227,0.543,-0.198,0.281
3,37.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,546.0,3.0,0.151,0.476,0.286,0.491,-0.095,0.448
4,24.0,-2.8,-0.4,-2.7,-0.2,25.0,10.0,374.0,1.0,0.195,0.551,0.428,0.676,-0.075,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,26.0,10.3,9.5,-0.2,-0.2,26.0,2.0,466.0,1.0,0.332,0.620,0.507,0.667,0.180,0.579
2898,20.0,-0.2,3.4,1.3,-0.1,1.0,20.0,375.0,3.0,-0.036,0.525,-0.001,0.564,-0.106,0.450
2899,36.0,-0.1,-1.8,-3.4,0.5,12.0,26.0,406.0,1.0,0.151,0.500,-0.057,0.469,0.346,0.529
2900,30.0,1.9,2.2,-3.4,-0.2,16.0,5.0,337.0,0.0,0.232,0.475,0.487,0.583,-0.169,0.304


In [183]:
y_pred = model_5.predict(X_test)

In [184]:
sq_resid = (y_pred - y_test)**2

In [185]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.647672409034977

In [186]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [189]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.582, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.616, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.647, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.607, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.645, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.582, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [190]:
grid.best_score_

0.6193625275279359

In [191]:
ys = grid.predict(X_test)

In [192]:
resids = (ys - y_test)**2

In [193]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.647672409034978

In [194]:
from sklearn.metrics import r2_score

In [195]:
r2_score(y_test, ys)

0.6277712268609108

In [197]:
!cd ..

In [202]:
!ls

2020GamesLeft.csv
MachineLearningModels.ipynb


In [203]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [209]:
allTheStats3.groupby(["FINAL_TEAM_NAME"]).count()

,Unnamed: 0,Year,Week_x,Abbr_RDBMS_Winner,Week2,Day,Date,Time,Home_Team,Opponent,...,ESTIM. WINS_FO_ANNUAL_Winner,SCHEDULE RANK_FO_ANNUAL_Winner,VAR._FO_ANNUAL_Winner,TOTAL_GAME_POINTS,TOTAL_GAME_YARDS,TOTAL_GAME_TURNOVERS,PLAYOFFS_BINARY,GAME_TYPE,FINAL_TEAM_NAME_Opp,Power Index DEF_Opp
FINAL_TEAM_NAME,,,,,,,,,,,,,,,,,,,,,
Arizona Cardinals,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Atlanta Falcons,94,94,94,94,94,94,94,94,94,94,...,94,94,94,94,94,94,94,94,94,94
Baltimore Ravens,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Buffalo Bills,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Carolina Panthers,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
Chicago Bears,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cincinnati Bengals,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cleveland Browns,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Dallas Cowboys,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92


In [211]:
elo = pd.read_csv("../Advanced Stats/538.csv")

In [217]:
fpi = pd.read_csv("../2015-2020_FPI.csv")

In [220]:
fpi=fpi.dropna(how="all")

In [219]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [223]:
#fpi["Ranks REM SOS"]
#fpi["Power Index TREND"]
def cleanme(c):
    if c == "--":
        return 0
    else:
        return int(c)

In [227]:
fpi["Ranks REM SOS"]=fpi["Ranks REM SOS"].apply(cleanme)

In [229]:
fpi["Power Index TREND"]=fpi["Power Index TREND"].apply(cleanme)

In [230]:
epa = pd.read_csv("../Advanced Stats/epa2015_2020.csv")

In [231]:
epa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TEAM              2936 non-null   object 
 1   Abbr              2936 non-null   object 
 2   EPA/play          2936 non-null   float64
 3   OVR_SUCCESS_RATE  2936 non-null   float64
 4   Dropback EPA      2936 non-null   float64
 5   DROPBACK_SR       2936 non-null   float64
 6   Rush EPA          2936 non-null   float64
 7   RUN_SR            2936 non-null   float64
 8   week              2936 non-null   int64  
 9   year              2936 non-null   int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 229.5+ KB


In [232]:
DVOA_Y = pd.read_csv("../Advanced Stats/DVOA_yearly.csv")

In [233]:
DVOA_W = pd.read_csv("../Advanced Stats/DVOA_weekly.csv")

In [237]:
DVOA_W=DVOA_W.iloc[:, :16]

In [239]:
DVOA_Y.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4'],
      dtype='object')

In [240]:
ycols = ['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'VAR.']

In [242]:
DVOA_Y = DVOA_Y.loc[:, ycols]

In [251]:
DVOA_Y.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4'],
      dtype='object')

In [260]:
fpi.columns

Index(['TEAM', 'Year ', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP'],
      dtype='object')

In [256]:
elo.columns

Index(['ELO WITH QB', 'ELO CHANGE', 'CURRENT QB ADJ', 'TEAM', 'DIVISION',
       'PROJ WINS', 'PROJ LOSES', 'POINT DIFF', 'MAKE PLAYOFFS',
       'WIN DIVISION', '1st-Round Bye', 'Win Superbowl', 'Before Week',
       'Year'],
      dtype='object')

In [254]:
DVOA_W.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR'],
      dtype='object')

In [277]:
remain = pd.read_csv("leftovers.csv")

In [262]:
remain.columns

Index(['Year', 'Week', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'TEAM_POINTS',
       'TEAM_YARDS', 'TEAM_TURNOVERS', 'TEAM_TODiff', 'GAMEID', 'Opponent',
       'Opponent_PTS', 'Opponent_Yds', 'Opponents_TO', 'Opponent_TODiff',
       'HOME_TEAM', 'BINARY_WIN'],
      dtype='object')

In [278]:
remain2 = pd.merge(remain, DVOA_Y, left_on=["FINAL_TEAM_NAME", "Year"], right_on=["Team", "YEAR"])

In [279]:
remain3 = pd.merge(remain2, DVOA_W, left_on=["FINAL_TEAM_NAME", "Year", "Week"], right_on=["Team", "YEAR", "WEEK"])

In [265]:
#remain4 = pd.merge(remain3, elo, left_on=["FINAL_TEAM_NAME", "Year", "Week"], right_on=["TEAM", "Year", "Before Week"])

In [280]:
remain5 = pd.merge(remain3, fpi, left_on=["FINAL_TEAM_NAME", "Year"], right_on=["TEAM", "Year "])

In [281]:
remain5.shape

(32, 74)

In [275]:
remain5.to_csv("remain22.csv", index=False)

In [276]:
remain5.columns

Index(['Year', 'Week', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'TEAM_POINTS',
       'TEAM_YARDS', 'TEAM_TURNOVERS', 'TEAM_TODiff', 'GAMEID', 'Opponent',
       'Opponent_PTS', 'Opponent_Yds', 'Opponents_TO', 'Opponent_TODiff',
       'HOME_TEAM', 'BINARY_WIN', 'RK_x', 'Team_x', 'Abbr_x', 'TOTAL DVOA_x',
       'LAST WEEK_x', 'DAVE_WEI_DVOA_x', 'RANK_x', 'W-L_x', 'OFFENSE DVOA_x',
       'OFF. RANK_x', 'DEFENSE DVOA_x', 'DEF. RANK_x', 'S.T. DVOA_x',
       'S.T. RANK_x', 'WEEK_x', 'YEAR_x', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4', 'RK_y', 'Team_y',
       'Abbr_y', 'TOTAL DVOA_y', 'LAST WEEK_y', 'DAVE_WEI_DVOA_y', 'RANK_y',
       'W-L_y', 'OFFENSE DVOA_y', 'OFF. RANK_y', 'DEFENSE DVOA_y',
       'DEF. RANK_y', 'S.T. DVOA_y', 'S.T. RANK_y', 'WEEK_y', 'YEAR_y', 'TEAM',
       'Year ', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power 

In [3]:
final_df3 = pd.read_csv("perfectlyCleanedData2.csv")

In [13]:
ty = final_df3.iloc[0, 21]

In [11]:
import re

In [21]:
regex = re.compile(".+/.+/")

In [39]:
regex.search("7/2/2000")#[0][:-1].replace("/", "-")

<re.Match object; span=(0, 4), match='7/2/'>

In [42]:
def cleanDate(d):
    match = regex.search(d)
    if match != None:
        match = match[0][:-1]
        match = match.replace("/", "-")
        return match
    else:
        return d

In [44]:
final_df3["W-L-T"]=final_df3["W-L-T"].apply(cleanDate)

In [46]:
final_df3.to_csv("perfectlyCleanedData3.csv", index=False)

## Redux

In [1]:
import pandas as pd, numpy as np, os, math

In [3]:
allTheStats = pd.read_csv("perfectlyCleanedData3.csv")

In [4]:
allTheStats.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [5]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [6]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [7]:
model_1 = sklearn.linear_model.LinearRegression()

In [8]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [10]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
#R_sq Score
model_1.score(X_train, y_train)

0.12837775897102

In [12]:
model_1.score(X_test, y_test)

0.07136027439629333

In [13]:
model_1.intercept_

array([22.78005983])

In [14]:
model_1.coef_

array([[0.69949756]])

In [15]:
y_pred = model_1.predict(X_test)

In [16]:
sq_resid = (y_pred - y_test)**2

In [17]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.321622897853451

In [18]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_1, params, verbose=3)

In [19]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.105, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.116, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.138, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.137, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.135, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.105, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [20]:
grid.best_score_

0.12621251574894304

In [21]:
ys = grid.predict(X_test)

In [22]:
resids = (ys - y_test)**2

In [23]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.321622897853449

In [24]:
from sklearn.metrics import r2_score

In [25]:
r2_score(y_test, ys)

0.07136027439629322

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [26]:
allTheStats.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [27]:
allTheStats2 = allTheStats.copy(deep=True)

In [28]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [29]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [30]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [31]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [32]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 183.2 KB


In [35]:
model_values.loc[:, ["TEAM_POINTS",'Power Index OFF', 'Power Index DEF_Opp']]

,TEAM_POINTS,Power Index OFF,Power Index DEF_Opp
0,24,0.9,-2.7
1,20,0.1,-2.4
2,34,0.1,-2.4
3,37,0.9,-2.7
4,24,-0.4,-2.7
...,...,...,...
2927,31,2.4,-2.2
2928,42,2.2,-2.2
2929,42,1.6,-2.2
2930,38,6.4,5.1


In [36]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [37]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [39]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [40]:
#R_sq Score
model_2.score(X_train, y_train)

0.20154513906522328

In [41]:
model_2.score(X_test, y_test)

0.21883076204876128

In [42]:
model_2.intercept_

array([24.05059979])

In [43]:
model_2.coef_

array([[ 0.09374783,  0.59316021, -0.92479362, -0.04941855,  0.03119231,
        -0.09964679]])

In [44]:
y_pred = model_2.predict(X_test)

In [45]:
sq_resid = (y_pred - y_test)**2

In [46]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.008383989531295

In [47]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [48]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.140, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.195, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.195, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.275, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.169, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.140, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [49]:
grid.best_score_

0.19516793987880796

In [50]:
ys = grid.predict(X_test)

In [51]:
resids = (ys - y_test)**2

In [52]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.008383989531295

In [53]:
from sklearn.metrics import r2_score

In [54]:
r2_score(y_test, ys)

0.21883076204876128

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [55]:
allTheStats2.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [56]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [57]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [58]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [59]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [60]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
 7   TEAM_YARDS           2932 non-null   int64  
 8   TEAM_TURNOVERS       2932 non-null   int64  
dtypes: float64(4), int64(5)
memory usage: 229.1 KB


In [61]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [62]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [64]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [65]:
#R_sq Score
model_3.score(X_train, y_train)

0.4844359762627359

In [66]:
model_3.score(X_test, y_test)

0.5343930244400317

In [67]:
model_3.intercept_

array([4.46879657])

In [68]:
model_3.coef_

array([[ 0.09629109,  0.07018035, -0.25159812,  0.17847358,  0.00694826,
        -0.10720859,  0.06490841, -1.87130443]])

In [69]:
y_pred = model_3.predict(X_test)

In [70]:
sq_resid = (y_pred - y_test)**2

In [71]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.36006083235951

In [72]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [73]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.541, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.488, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.445, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.479, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.440, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.541, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [74]:
grid.best_score_

0.47870269881686944

In [75]:
ys = grid.predict(X_test)

In [76]:
resids = (ys - y_test)**2

In [77]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.36006083235951

In [78]:
from sklearn.metrics import r2_score

In [79]:
r2_score(y_test, ys)

0.5343930244400317

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [80]:
allTheStats3.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [81]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [82]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
 7   TEAM_YARDS           2932 non-null   int64  
 8   TEAM_TURNOVERS       2932 non-null   int64  
dtypes: float64(4), int64(5)
memory usage: 229.1 KB


In [83]:
from sklearn.ensemble import RandomForestClassifier

In [84]:
rf = RandomForestClassifier(n_estimators=4000, criterion="entropy", max_depth=5)

In [85]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [87]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
#R_sq Score
rf.score(X_train, y_train)

0.24010914051841747

In [89]:
rf.score(X_test, y_test)

0.07503410641200546

In [90]:
rf.classes_

array([ 0,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55,
       57], dtype=int64)

In [91]:
y_pred = rf.predict(X_test)

In [92]:
sq_resid = (y_pred - y_test)**2

In [93]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.319375606051864

In [94]:
## STOP POINT FOR RF ##

In [95]:
from sklearn.model_selection import RandomizedSearchCV

In [96]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [97]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=40, max_depth=10, criterion=entropy, score=0.061, total=   0.2s
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=40, max_depth=10, criterion=entropy, score=0.075, total=   0.2s
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


...

[CV]  n_estimators=8000, max_depth=2, criterion=entropy, score=0.109, total=  14.3s
[CV] n_estimators=8000, max_depth=2, criterion=entropy ...............


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=8000, max_depth=2, criterion=entropy, score=0.082, total=  14.2s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.070, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.077, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.091, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.114, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.075, total=   0.0s


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [98]:
grid.best_score_

0.09594533029612755

In [166]:
ys = grid.predict(X_test)

In [167]:
resids = (ys - y_test)**2

In [168]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.918588590639681

In [169]:
from sklearn.metrics import r2_score

In [170]:
r2_score(y_test, ys)

0.214525684460524

In [99]:
rf2 = RandomForestClassifier(n_estimators=40, criterion="gini", max_depth=5)

In [100]:
rf2.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [101]:
rf2.score(X_train, y_train)

0.24874943155979992

In [102]:
rf2.score(X_test, y_test)

0.07776261937244201

In [103]:
y_pred = rf2.predict(X_test)

In [104]:
sq_resid = (y_pred - y_test)**2

In [105]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.431611292991295

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [106]:
allTheStats3.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [107]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [108]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2932 non-null   int64  
 1   Power Index FPI                2932 non-null   float64
 2   Power Index OFF                2932 non-null   float64
 3   Power Index DEF_Opp            2932 non-null   float64
 4   Power Index ST                 2932 non-null   float64
 5   Ranks SOS                      2932 non-null   int64  
 6   Ranks AVGWP                    2932 non-null   int64  
 7   TEAM_YARDS                     2932 non-null   int64  
 8   TEAM_TURNOVERS                 2932 non-null   int64  
 9   EPA/play_RDBMS_Winner          2932 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2932 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2932 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2932 non-null   f

In [109]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [110]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [112]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [113]:
#R_sq Score
model_5.score(X_train, y_train)

0.6248639025345639

In [114]:
model_5.score(X_test, y_test)

0.612299807470251

In [115]:
model_5.intercept_

array([19.25145464])

In [116]:
model_5.coef_

array([[ 1.37686281e-01, -7.08405747e-02, -6.34908984e-02,
         6.95595163e-01,  3.26944863e-02, -4.09261346e-02,
         2.15664307e-02,  5.02436037e-01,  2.74083846e+01,
        -2.43643998e+01,  4.94395420e+00,  9.57538835e+00,
         2.60905471e+00,  4.11135591e+00]])

In [117]:
model_values

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_YARDS,TEAM_TURNOVERS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
0,24,-2.0,0.9,-2.7,-0.5,15,13,349,0,0.180,0.468,0.257,0.500,-0.041,0.375
1,20,-2.6,0.1,-2.4,0.0,12,27,345,3,-0.008,0.442,-0.009,0.483,-0.007,0.391
2,34,-2.6,0.1,-2.4,0.0,12,27,444,1,0.053,0.436,0.227,0.543,-0.198,0.281
3,37,-2.0,0.9,-2.7,-0.5,15,13,546,3,0.151,0.476,0.286,0.491,-0.095,0.448
4,24,-2.8,-0.4,-2.7,-0.2,25,10,374,1,0.195,0.551,0.428,0.676,-0.075,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927,31,1.9,2.4,-2.2,-0.5,9,17,444,1,0.057,0.453,0.204,0.484,-0.081,0.424
2928,42,1.9,2.2,-2.2,-0.2,16,5,334,0,0.030,0.514,0.018,0.560,0.060,0.400
2929,42,-2.4,1.6,-2.2,0.2,7,27,607,2,0.299,0.478,0.567,0.581,-0.144,0.308
2930,38,4.7,6.4,5.1,-0.4,2,3,324,0,-0.314,0.339,-0.366,0.357,-0.205,0.300


In [118]:
y_pred = model_5.predict(X_test)

In [119]:
sq_resid = (y_pred - y_test)**2

In [120]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.7839731537685095

In [123]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_5, params, verbose=3)

In [124]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.634, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.625, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.586, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.615, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.634, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.634, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [125]:
grid.best_score_

0.6188779739484955

In [126]:
ys = grid.predict(X_test)

In [127]:
resids = (ys - y_test)**2

In [128]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.7839731537685095

In [129]:
from sklearn.metrics import r2_score

In [130]:
r2_score(y_test, ys)

0.612299807470251

In [203]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [2]:
import pandas as pd, numpy as np, os, math

In [3]:
outstanding_games = pd.read_csv("2020GamesLeft.csv")

In [131]:
allTheStats = pd.read_csv("../perfectlyCleaned_Data.csv")

In [132]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [133]:
outstanding_games.columns

NameError: name 'outstanding_games' is not defined

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [134]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [135]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [136]:
model_1 = sklearn.linear_model.LinearRegression()

In [137]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [144]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [145]:
#R_sq Score 11.16
model_1.score(X_train, y_train)

0.11185919136742972

In [146]:
#R_sq 12.10
model_1.score(X_test, y_test)

0.1236776812910152

In [147]:
model_1.intercept_

array([22.76800928])

In [148]:
model_1.coef_

array([[0.6385831]])

In [149]:
y_pred = model_1.predict(X_test)

In [150]:
sq_resid = (y_pred - y_test)**2

In [151]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.527513175375787

In [161]:
X = X.reshape(-1, 1)

In [162]:
y_prediction1 = model_1.predict(X)

In [171]:
predictions = pd.Series(y_prediction1.reshape(-1), name="Model1_Prediction")

In [173]:
model_predictions_1 = pd.concat([allTheStats.loc[:, ["GAMEID", "FINAL_TEAM_NAME"]], predictions], axis=1)

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [174]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [182]:
allTheStats2 = allTheStats.copy(deep=True)

In [44]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [195]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [196]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [197]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [198]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
dtypes: float64(7)
memory usage: 181.4 KB


In [188]:
model_values.loc[:, ["FINAL_TEAM_NAME", 'Power Index OFF', 'Power Index DEF_Opp']]

,FINAL_TEAM_NAME,Power Index OFF,Power Index DEF_Opp
0,Detroit Lions,0.9,-2.7
1,Chicago Bears,0.1,-2.4
2,Chicago Bears,0.1,-2.4
3,Detroit Lions,0.9,-2.7
4,Washington Redskins,-0.4,-2.7
...,...,...,...
2897,Kansas City Chiefs,9.5,-0.2
2898,Oakland Raiders,3.4,1.3
2899,New England Patriots,-1.8,-3.4
2900,Buffalo Bills,2.2,-3.4


In [199]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [200]:
model_values.shape

(2902, 7)

In [231]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [210]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [211]:
#R_sq Score
model_2.score(X_train, y_train)

0.2038363076355726

In [212]:
model_2.score(X_test, y_test)

0.21541019805453554

In [213]:
model_2.intercept_

array([24.54785568])

In [214]:
model_2.coef_

array([[ 0.06087359,  0.58311624, -1.00594342,  0.65866892, -0.01333301,
        -0.09084559]])

In [215]:
y_pred = model_2.predict(X_test)

In [216]:
sq_resid = (y_pred - y_test)**2

In [217]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.1567476109864545

In [218]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [221]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.184, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.233, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.246, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.202, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.118, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.184, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [222]:
grid.best_score_

0.1964905900946871

In [223]:
ys = grid.predict(X_test)

In [224]:
resids = (ys - y_test)**2

In [225]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.1567476109864545

In [226]:
from sklearn.metrics import r2_score

In [227]:
r2_score(y_test, ys)

0.21541019805453554

In [232]:
X

array([[-2. ,  0.9, -2.7, -0.5, 15. , 13. ],
       [-2.6,  0.1, -2.4,  0. , 12. , 27. ],
       [-2.6,  0.1, -2.4,  0. , 12. , 27. ],
       ...,
       [-0.1, -1.8, -3.4,  0.5, 12. , 26. ],
       [ 1.9,  2.2, -3.4, -0.2, 16. ,  5. ],
       [-0.2,  3.4, -0.2, -0.1,  1. , 20. ]])

In [229]:
X = X.reshape(-1, 1)

In [233]:
y_prediction2 = model_2.predict(X)

In [240]:
predictions_2 = pd.Series(y_prediction2.reshape(-1), name="Model2_Prediction")

In [241]:
model_predictions_2 = pd.concat([allTheStats.loc[:, ["GAMEID", "FINAL_TEAM_NAME"]], predictions_2], axis=1)

In [242]:
model_predictions_2

,GAMEID,FINAL_TEAM_NAME,Model2_Prediction
0,1.0,Detroit Lions,25.956638
1,1.0,Chicago Bears,24.249333
2,2.0,Chicago Bears,24.249333
3,2.0,Detroit Lions,25.956638
4,3.0,Washington Redskins,25.486695
...,...,...,...
2897,1449.0,Kansas City Chiefs,30.255563
2898,1450.0,Oakland Raiders,23.314438
2899,1450.0,New England Patriots,24.719720
2900,1451.0,Buffalo Bills,28.567289


In [237]:
model_predictions_1

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction
0,1.0,Detroit Lions,21.490843
1,1.0,Chicago Bears,21.107693
2,2.0,Chicago Bears,21.107693
3,2.0,Detroit Lions,21.490843
4,3.0,Washington Redskins,20.979977
...,...,...,...
2897,1449.0,Kansas City Chiefs,29.345415
2898,1450.0,Oakland Raiders,22.640293
2899,1450.0,New England Patriots,22.704151
2900,1451.0,Buffalo Bills,23.981317


In [247]:
predictions_12 = pd.concat([model_predictions_1, model_predictions_2.Model2_Prediction], axis=1)

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [248]:
allTheStats2.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [249]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [250]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [251]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [252]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [253]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [254]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [286]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [292]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [293]:
#R_sq Score
model_3.score(X_train, y_train)

0.4877758699207122

In [294]:
model_3.score(X_test, y_test)

0.5308093754462411

In [295]:
model_3.intercept_

array([4.27564302])

In [296]:
model_3.coef_

array([[ 0.11208271,  0.06262227, -0.2948755 ,  0.76401068,  0.01230253,
        -0.09257412,  0.06451639, -1.82982766]])

In [297]:
y_pred = model_3.predict(X_test)

In [298]:
sq_resid = (y_pred - y_test)**2

In [299]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.409170175986392

In [300]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [301]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.465, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.461, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.473, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.550, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.463, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.465, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [302]:
grid.best_score_

0.48236867019506774

In [303]:
ys = grid.predict(X_test)

In [304]:
resids = (ys - y_test)**2

In [305]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.409170175986392

In [306]:
from sklearn.metrics import r2_score

In [307]:
r2_score(y_test, ys)

0.5308093754462408

In [308]:
X.shape

(2902, 8)

In [284]:
X = X.reshape(-1, 1)

In [310]:
y_prediction3 = model_3.predict(X)

In [313]:
predictions_3 = pd.Series(y_prediction3.reshape(-1), name="Model3_Prediction")

In [314]:
model_predictions_3 = pd.concat([allTheStats.loc[:, ["GAMEID", "FINAL_TEAM_NAME"]], predictions_3], axis=1)

In [315]:
model_predictions_3

,GAMEID,FINAL_TEAM_NAME,Model3_Prediction
0,1.0,Detroit Lions,26.019291
1,1.0,Chicago Bears,19.114992
2,2.0,Chicago Bears,29.161770
3,2.0,Detroit Lions,33.239537
4,3.0,Washington Redskins,26.261249
...,...,...,...
2897,1449.0,Kansas City Chiefs,34.300707
2898,1450.0,Oakland Raiders,20.871386
2899,1450.0,New England Patriots,27.640827
2900,1451.0,Buffalo Bills,26.952137


In [237]:
model_predictions_1

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction
0,1.0,Detroit Lions,21.490843
1,1.0,Chicago Bears,21.107693
2,2.0,Chicago Bears,21.107693
3,2.0,Detroit Lions,21.490843
4,3.0,Washington Redskins,20.979977
...,...,...,...
2897,1449.0,Kansas City Chiefs,29.345415
2898,1450.0,Oakland Raiders,22.640293
2899,1450.0,New England Patriots,22.704151
2900,1451.0,Buffalo Bills,23.981317


In [316]:
predictions_123 = pd.concat([predictions_12 , model_predictions_3.Model3_Prediction], axis=1)

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [136]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [318]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [319]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [320]:
from sklearn.ensemble import RandomForestClassifier

In [321]:
rf = RandomForestClassifier(n_estimators=4000, criterion="gini", max_depth=5)

In [322]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [323]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [324]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [325]:
#R_sq Score
rf.score(X_train, y_train)

0.26608455882352944

In [326]:
rf.score(X_test, y_test)

0.081267217630854

In [327]:
rf.classes_

array([ 0.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 51., 52., 54., 55., 57.])

In [328]:
y_pred = rf.predict(X_test)

In [329]:
sq_resid = (y_pred - y_test)**2

In [330]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.538559904074555

In [331]:
## STOP POINT FOR RF ##

In [332]:
from sklearn.model_selection import RandomizedSearchCV

In [333]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [334]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=400, max_depth=10, criterion=gini ..................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=400, max_depth=10, criterion=gini, score=0.101, total=   1.1s
[CV] n_estimators=400, max_depth=10, criterion=gini ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=400, max_depth=10, criterion=gini, score=0.097, total=   1.1s
[CV] n_estimators=400, max_depth=10, criterion=gini ..................


...

[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.101, total=  26.7s
[CV] n_estimators=40, max_depth=2, criterion=entropy .................
[CV]  n_estimators=40, max_depth=2, criterion=entropy, score=0.083, total=   0.1s
[CV] n_estimators=40, max_depth=2, criterion=entropy .................
[CV]  n_estimators=40, max_depth=2, criterion=entropy, score=0.094, total=   0.1s
[CV] n_estimators=40, max_depth=2, criterion=entropy .................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=40, max_depth=2, criterion=entropy, score=0.087, total=   0.1s
[CV] n_estimators=40, max_depth=2, criterion=entropy .................
[CV]  n_estimators=40, max_depth=2, criterion=entropy, score=0.083, total=   0.1s
[CV] n_estimators=40, max_depth=2, criterion=entropy .................
[CV]  n_estimators=40, max_depth=2, criterion=entropy, score=0.078, total=   0.1s


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  9.0min finished
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [335]:
grid.best_score_

0.09972687967942634

In [336]:
ys = grid.predict(X_test)

In [337]:
resids = (ys - y_test)**2

In [338]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.510867502978698

In [339]:
from sklearn.metrics import r2_score

In [340]:
r2_score(y_test, ys)

0.37699158342636263

In [341]:
X

array([[-2.00e+00,  9.00e-01, -2.70e+00, ...,  1.30e+01,  3.49e+02,
         0.00e+00],
       [-2.60e+00,  1.00e-01, -2.40e+00, ...,  2.70e+01,  3.45e+02,
         3.00e+00],
       [-2.60e+00,  1.00e-01, -2.40e+00, ...,  2.70e+01,  4.44e+02,
         1.00e+00],
       ...,
       [-1.00e-01, -1.80e+00, -3.40e+00, ...,  2.60e+01,  4.06e+02,
         1.00e+00],
       [ 1.90e+00,  2.20e+00, -3.40e+00, ...,  5.00e+00,  3.37e+02,
         0.00e+00],
       [-2.00e-01,  3.40e+00, -2.00e-01, ...,  2.00e+01,  3.83e+02,
         2.00e+00]])

In [229]:
#X = X.reshape(-1, 1)

In [342]:
y_prediction4 = rf.predict(X)

In [343]:
predictions_4 = pd.Series(y_prediction4.reshape(-1), name="Model4_Prediction")

In [344]:
model_predictions_4 = pd.concat([predictions_123, predictions_4], axis=1)

In [345]:
model_predictions_4

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction,Model2_Prediction,Model3_Prediction,Model4_Prediction
0,1.0,Detroit Lions,21.490843,25.956638,26.019291,27.0
1,1.0,Chicago Bears,21.107693,24.249333,19.114992,20.0
2,2.0,Chicago Bears,21.107693,24.249333,29.161770,27.0
3,2.0,Detroit Lions,21.490843,25.956638,33.239537,24.0
4,3.0,Washington Redskins,20.979977,25.486695,26.261249,24.0
...,...,...,...,...,...,...
2897,1449.0,Kansas City Chiefs,29.345415,30.255563,34.300707,31.0
2898,1450.0,Oakland Raiders,22.640293,23.314438,20.871386,20.0
2899,1450.0,New England Patriots,22.704151,24.719720,27.640827,27.0
2900,1451.0,Buffalo Bills,23.981317,28.567289,26.952137,27.0


In [237]:
model_predictions_1

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction
0,1.0,Detroit Lions,21.490843
1,1.0,Chicago Bears,21.107693
2,2.0,Chicago Bears,21.107693
3,2.0,Detroit Lions,21.490843
4,3.0,Washington Redskins,20.979977
...,...,...,...
2897,1449.0,Kansas City Chiefs,29.345415
2898,1450.0,Oakland Raiders,22.640293
2899,1450.0,New England Patriots,22.704151
2900,1451.0,Buffalo Bills,23.981317


In [247]:
predictions_12 = pd.concat([model_predictions_1, model_predictions_2.Model2_Prediction], axis=1)

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [346]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [347]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [348]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2902 non-null   float64
 1   Power Index FPI                2902 non-null   float64
 2   Power Index OFF                2902 non-null   float64
 3   Power Index DEF_Opp            2902 non-null   float64
 4   Power Index ST                 2902 non-null   float64
 5   Ranks SOS                      2902 non-null   float64
 6   Ranks AVGWP                    2902 non-null   float64
 7   TEAM_YARDS                     2902 non-null   float64
 8   TEAM_TURNOVERS                 2902 non-null   float64
 9   EPA/play_RDBMS_Winner          2902 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2902 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2902 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2902 non-null   f

In [360]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [361]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [362]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [363]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [364]:
#R_sq Score
model_5.score(X_train, y_train)

0.6332710872023117

In [365]:
model_5.score(X_test, y_test)

0.6175452027881858

In [366]:
model_5.intercept_

array([19.78658768])

In [367]:
model_5.coef_

array([[ 1.24015377e-01, -9.05685382e-02, -5.98345513e-02,
         1.00656027e+00,  1.31475637e-02, -5.18599953e-02,
         1.86220891e-02,  5.16350336e-01,  2.76937189e+01,
        -2.74690060e+01,  5.27177496e+00,  1.32151334e+01,
         2.44318148e+00,  5.33981219e+00]])

In [368]:
y_pred = model_5.predict(X_test)

In [369]:
sq_resid = (y_pred - y_test)**2

In [370]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.596824428454162

In [371]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_5, params, verbose=3)

In [372]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.653, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.626, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.606, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.625, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.623, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.653, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [373]:
grid.best_score_

0.6265894469232373

In [374]:
ys = grid.predict(X_test)

In [375]:
resids = (ys - y_test)**2

In [376]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.596824428454162

In [377]:
from sklearn.metrics import r2_score

In [378]:
r2_score(y_test, ys)

0.6175452027881858

In [379]:
X

array([[-2.   ,  0.9  , -2.7  , ...,  0.5  , -0.041,  0.375],
       [-2.6  ,  0.1  , -2.4  , ...,  0.483, -0.007,  0.391],
       [-2.6  ,  0.1  , -2.4  , ...,  0.543, -0.198,  0.281],
       ...,
       [-0.1  , -1.8  , -3.4  , ...,  0.469,  0.346,  0.529],
       [ 1.9  ,  2.2  , -3.4  , ...,  0.583, -0.169,  0.304],
       [-0.2  ,  3.4  , -0.2  , ...,  0.538, -0.429,  0.35 ]])

In [380]:
y_prediction5 = model_5.predict(X)

In [381]:
predictions_5 = pd.Series(y_prediction5.reshape(-1), name="Model5_Prediction")

In [382]:
model_predictions_fin = pd.concat([model_predictions_4, predictions_5], axis=1)

In [383]:
model_predictions_fin

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction,Model2_Prediction,Model3_Prediction,Model4_Prediction,Model5_Prediction
0,1.0,Detroit Lions,21.490843,25.956638,26.019291,27.0,27.131508
1,1.0,Chicago Bears,21.107693,24.249333,19.114992,20.0,22.373294
2,2.0,Chicago Bears,21.107693,24.249333,29.161770,27.0,26.021331
3,2.0,Detroit Lions,21.490843,25.956638,33.239537,24.0,31.618060
4,3.0,Washington Redskins,20.979977,25.486695,26.261249,24.0,30.166242
...,...,...,...,...,...,...,...
2897,1449.0,Kansas City Chiefs,29.345415,30.255563,34.300707,31.0,36.628899
2898,1450.0,Oakland Raiders,22.640293,23.314438,20.871386,20.0,20.957492
2899,1450.0,New England Patriots,22.704151,24.719720,27.640827,27.0,27.545013
2900,1451.0,Buffalo Bills,23.981317,28.567289,26.952137,27.0,30.911144


In [402]:
data_df = pd.read_csv("perfectlyCleanedData3.csv")

In [403]:
tampas = data_df[(data_df.FINAL_TEAM_NAME.str.startswith("Tampa Bay Buc") & (data_df.Year == 2020)) | (data_df.Opponent.str.startswith("Tampa Bay Buc")) & (data_df.Year == 2020)]

In [404]:
tampas.shape

(16, 73)

In [405]:
tenn = data_df[(data_df.FINAL_TEAM_NAME.str.startswith("Tenn") & (data_df.Year == 2020)) | (data_df.Opponent.str.startswith("Tenn")) & (data_df.Year == 2020)]

In [406]:
tenn.shape

(16, 73)

In [408]:
missing_games = pd.concat([tampas, tenn])

In [409]:
missing_games.shape

(32, 73)

In [411]:
missing_games = missing_games.sort_values(["GAMEID"])

In [422]:
opponentDef

,FINAL_TEAM_NAME,Power Index DEF,GAMEID
0,Detroit Lions,-2.4,1.0
1,Chicago Bears,-2.7,1.0
2,Chicago Bears,-2.7,2.0
3,Detroit Lions,-2.4,2.0
4,Washington Redskins,-2.2,3.0
...,...,...,...
2897,Kansas City Chiefs,1.0,1449.0
2898,Oakland Raiders,-3.4,1450.0
2899,New England Patriots,1.3,1450.0
2900,Buffalo Bills,-0.2,1451.0


In [423]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "Year"]]

In [441]:
defDf = defDf.drop(columns=['FINAL_TEAM_NAME_Opp'])

In [442]:
defDf.shape

(32, 74)

In [435]:
new_df = opponentDef.drop_duplicates()

In [436]:
defDf = pd.merge(missing_games, new_df, "left", left_on=["Opponent", "Year"], right_on=["FINAL_TEAM_NAME", "Year"], suffixes=("", "_Opp"))

In [ ]:
def_Df pd.merge(missing_games, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [443]:
defDf.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [445]:
#['TEAM_POINTS', 'Power Index FPI']

In [446]:
#['TEAM_POINTS', 'Power Index FPI', Power Index OFF', 'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',]

In [447]:
#['TEAM_POINTS', 'Power Index FPI', Power Index OFF', 'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
#'TEAM_TURNOVERS', 'TEAM_YARDS', EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner','DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']

In [448]:
missing = defDf.loc[:, ['TEAM_POINTS', 'Power Index FPI', 'Power Index OFF', 'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP', 'TEAM_TURNOVERS', 'TEAM_YARDS', 'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner','DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [449]:
models = [model_1, model_2, model_3, model_5, rf]

In [450]:
Y = missing.iloc[:, 0].values

In [477]:
for i, r in defDf.iterrows():
    if r["BINARY_WIN"] == 0:
        p = r["TEAM_POINTS"]
        o = r["OPPONENTS_POINTS"]
        defDf.loc[i, "OPPONENTS_POINTS"] = p
        defDf.loc[i, "TEAM_POINTS"] = o
    else:
        next

In [483]:
defDf2 = defDf.loc[:, ['TEAM_POINTS', 'Power Index FPI', 'Power Index OFF', 'Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP', 'TEAM_TURNOVERS', 'TEAM_YARDS', 'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner','DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [482]:
Y = defDf2.iloc[:, 0].values

In [501]:
defDf.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF_Opp']]

,FINAL_TEAM_NAME,Power Index DEF_Opp
0,New Orleans Saints,NaN
1,Tampa Bay Buccaneers,-0.2
2,Tennessee Titans,3.4
3,Denver Broncos,NaN
4,Tennessee Titans,-4.5
5,Jacksonville Jaguars,NaN
6,Carolina Panthers,NaN
7,Tampa Bay Buccaneers,-0.9
8,Minnesota Vikings,NaN
9,Tennessee Titans,0.0


In [545]:
new_Df = new_Df.drop(columns=["Year"])

In [564]:
new_Df

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_TURNOVERS,TEAM_YARDS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
326,24,1.5,3.6,4.3,0.0,30,16,3,362,0.052,0.379,0.185,0.400,-0.151,0.348
327,20,1.5,3.6,-2.9,0.0,30,16,0,367,0.231,0.597,0.103,0.500,0.388,0.714
328,23,5.8,1.2,-0.2,-0.5,14,11,0,271,-0.018,0.464,-0.035,0.455,0.014,0.480
329,3,5.8,1.2,-0.2,-0.5,14,11,2,420,-0.479,0.367,-0.477,0.386,-0.494,0.200
330,19,5.8,1.2,3.7,-0.5,14,11,1,243,-0.023,0.348,-0.048,0.347,0.037,0.350
331,24,1.5,3.6,3.7,0.0,30,16,0,228,-0.095,0.302,-0.041,0.360,-0.144,0.250
332,17,-2.1,-5.6,-2.2,-0.2,10,25,0,228,-0.141,0.397,-0.091,0.441,-0.295,0.263
333,14,-2.7,-6.2,-2.2,0.1,5,28,0,377,-0.085,0.407,-0.179,0.441,0.043,0.360
334,30,-8.8,-4.6,-2.2,0.3,27,31,0,354,0.266,0.560,0.293,0.620,0.210,0.440
335,30,1.9,2.4,-2.2,-0.5,9,17,1,444,0.057,0.453,0.204,0.484,-0.081,0.424


In [566]:
new_Df = new_Df.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [567]:
Y = new_Df.iloc[:, 0].values
X1 = new_Df.iloc[:, 1].values
X2 = new_Df.iloc[:, 1:7].values
X3 = new_Df.iloc[:, 1:9].values
X4 = new_Df.iloc[:, 1:9].values
X5 = new_Df.iloc[:, 1:].values

In [568]:
y1 = model_1.predict(X1.reshape(-1, 1))

In [569]:
y2 = model_2.predict(X2)

In [570]:
y3 = model_3.predict(X3)

In [571]:
y4 = rf.predict(X4)

In [572]:
y5 = model_5.predict(X5)

In [573]:
y5

array([[26.02417576],
       [28.12845021],
       [19.74765573],
       [ 7.47448453],
       [20.42341903],
       [17.83357653],
       [14.26623987],
       [19.25751244],
       [29.46949974],
       [26.12730592],
       [22.41375377],
       [37.81528102],
       [27.42984713],
       [33.07602329],
       [12.38589724],
       [16.8461951 ],
       [10.86791669],
       [21.64092848],
       [18.26945774],
       [32.53187444],
       [16.9660278 ],
       [27.57901436],
       [26.68510344],
       [34.46634979],
       [23.56900961],
       [33.1563647 ],
       [41.81128779],
       [25.17629696],
       [29.89762679],
       [25.6623598 ]])

In [584]:
missing_preds = pd.concat([pd.Series(y1.reshape(-1), name="Model1_Prediction"),pd.Series(y2.reshape(-1), name="Model2_Prediction"),pd.Series(y3.reshape(-1), name="Model3_Prediction"),pd.Series(y4.reshape(-1), name="Model4_Prediction"),pd.Series(y5.reshape(-1), name="Model5_Prediction")], axis=1)

In [592]:
new_Df.shape

(30, 76)

In [614]:
new_Df = new_Df.reset_index(drop=True)

In [615]:
missing_ones = pd.concat([new_Df, missing_preds], axis=1)

In [616]:
missing_ones

,Year,Week_x,Abbr_RDBMS_Winner,Week2,Day,Date,Time,Home_Team,FINAL_TEAM_NAME,Opponent,...,PLAYOFFS_BINARY,GAME_TYPE,Year,TEAM,Power Index DEF_Opp,Model1_Prediction,Model2_Prediction,Model3_Prediction,Model4_Prediction,Model5_Prediction
0,2020,7,TEN,9,Sun,25-Oct,1:00:00 PM,1,Tennessee Titans,Pittsburgh Steelers,...,0,Regular Season,2020,Pittsburgh Steelers,4.3,23.725884,20.559308,20.154583,24.0,26.024176
1,2020,8,TEN,9,Sun,1-Nov,1:00:00 PM,0,Tennessee Titans,Cincinnati Bengals,...,0,Regular Season,2020,Cincinnati Bengals,-2.9,23.725884,27.802101,28.089752,27.0,28.128450
2,2020,1,TB,9,Sun,13-Sep,4:25:00 PM,0,Tampa Bay Buccaneers,New Orleans Saints,...,0,Regular Season,2020,New Orleans Saints,-0.2,26.471791,24.286553,21.315701,20.0,19.747656
3,2020,9,TB,9,Sun,8-Nov,8:20:00 PM,1,Tampa Bay Buccaneers,New Orleans Saints,...,0,Regular Season,2020,New Orleans Saints,-0.2,26.471791,24.286553,27.268988,24.0,7.474485
4,2020,5,TB,9,Thu,8-Oct,8:20:00 PM,0,Tampa Bay Buccaneers,Chicago Bears,...,0,Regular Season,2020,Chicago Bears,3.7,26.471791,20.363373,16.529400,20.0,20.423419
5,2020,9,TEN,9,Sun,8-Nov,1:00:00 PM,1,Tennessee Titans,Chicago Bears,...,0,Regular Season,2020,Chicago Bears,3.7,23.725884,21.162874,17.175795,10.0,17.833577
6,2020,9,CHI,9,Sun,8-Nov,1:00:00 PM,0,Chicago Bears,Tennessee Titans,...,0,Regular Season,2020,Tennessee Titans,-2.2,21.426985,20.831442,16.703918,17.0,14.266240
7,2020,1,DEN,9,Mon,14-Sep,10:10:00 PM,1,Denver Broncos,Tennessee Titans,...,0,Regular Season,2020,Tennessee Titans,-2.2,21.043835,20.436777,26.102005,27.0,19.257512
8,2020,2,JAX,9,Sun,20-Sep,1:00:00 PM,0,Jacksonville Jaguars,Tennessee Titans,...,0,Regular Season,2020,Tennessee Titans,-2.2,17.148478,20.564305,24.180355,27.0,29.469500
9,2020,3,MIN,9,Sun,27-Sep,1:00:00 PM,1,Minnesota Vikings,Tennessee Titans,...,0,Regular Season,2020,Tennessee Titans,-2.2,23.981317,26.282363,30.258027,31.0,26.127306


In [579]:
stats2 = pd.read_csv("perfectlyCleanedData4.csv")

In [577]:
stats2=stats2.loc[:, ["Year", 'TEAM_POINTS', 'Power Index FPI', 'Power Index OFF', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP', 'TEAM_TURNOVERS', 'TEAM_YARDS', 'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner','DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [517]:
fpi = pd.read_csv("../2015-2020_FPI.csv")

In [518]:
fpi

,TEAM,Year,W-L-T,Power Index FPI,Power Index RK,Power Index TREND,Power Index OFF,Power Index DEF,Power Index ST,Ranks SOS,Ranks REM SOS,Ranks AVGWP
0,Seattle Seahawks,2015,11/7/2000,8.8,1,--,6.0,3.0,-0.3,6,--,6
1,Arizona Cardinals,2015,14-4-0,8.3,2,--,6.0,2.6,-0.4,17,--,3
2,Carolina Panthers,2015,17-2-0,7.9,3,--,3.5,4.8,-0.4,28,--,1
3,New England Patriots,2015,13-5-0,7.2,4,--,5.6,1.2,0.4,18,--,2
4,Pittsburgh Steelers,2015,11/7/2000,7.1,5,--,4.9,1.9,0.3,5,--,9
...,...,...,...,...,...,...,...,...,...,...,...,...
187,Detroit Lions,2020,3/5/2000,-4.7,28,--,-0.9,-3.9,0.1,4,16,23
188,New York Giants,2020,2/7/2000,-5.4,29,--,-3.6,-1.9,0.1,11,18,22
189,Dallas Cowboys,2020,2/7/2000,-7.8,30,--,-5.0,-2.3,-0.5,17,24,30
190,Jacksonville Jaguars,2020,1/7/2000,-8.8,31,--,-4.6,-4.5,0.3,27,2,31


In [580]:
missing = stats2[(stats2.Opponent.str.startswith("Tampa") | stats2.Opponent.str.startswith("Tenn")) | (stats2.FINAL_TEAM_NAME.str.startswith("Tampa") | stats2.FINAL_TEAM_NAME.str.startswith("Tenn"))]

In [526]:
fpi = fpi.loc[:, ["Year ", "TEAM", "Power Index DEF"]]

In [588]:
new_Df = pd.merge(missing, fpi, left_on=["Opponent", "Year"], right_on=["TEAM", "Year "], suffixes=("", "_Opp"))

In [538]:
new_Df

,Year,Week_x,Abbr_RDBMS_Winner,Week2,Day,Date,Time,Home_Team,FINAL_TEAM_NAME,Opponent,...,SCHEDULE RANK_FO_ANNUAL_Winner,VAR._FO_ANNUAL_Winner,TOTAL_GAME_POINTS,TOTAL_GAME_YARDS,TOTAL_GAME_TURNOVERS,PLAYOFFS_BINARY,GAME_TYPE,Year,TEAM,Power Index DEF_Opp
0,2015,11,PHI,11,Sun,22-Nov,1:02PM,0,Philadelphia Eagles,Tampa Bay Buccaneers,...,-0.007,0.195,62,904,5,0,Regular Season,2015,Tampa Bay Buccaneers,-2.8
1,2015,2,NO,2,Sun,20-Sep,1:02PM,0,New Orleans Saints,Tampa Bay Buccaneers,...,-0.035,0.128,45,656,5,0,Regular Season,2015,Tampa Bay Buccaneers,-2.8
2,2015,10,DAL,10,Sun,15-Nov,1:05PM,1,Dallas Cowboys,Tampa Bay Buccaneers,...,0.005,0.045,16,546,3,0,Regular Season,2015,Tampa Bay Buccaneers,-2.8
3,2015,8,ATL,8,Sun,1-Nov,1:02PM,0,Atlanta Falcons,Tampa Bay Buccaneers,...,-0.074,0.072,43,786,4,0,Regular Season,2015,Tampa Bay Buccaneers,-2.8
4,2015,13,ATL,13,Sun,6-Dec,1:03PM,1,Atlanta Falcons,Tampa Bay Buccaneers,...,-0.074,0.072,42,707,3,0,Regular Season,2015,Tampa Bay Buccaneers,-2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2020,5,TEN,9,Tue,13-Oct,7:00:00 PM,1,Tennessee Titans,Buffalo Bills,...,-0.048,0.069,58,704,3,0,Regular Season,2020,Buffalo Bills,-0.2
352,2020,6,TEN,9,Sun,18-Oct,1:00:00 PM,1,Tennessee Titans,Houston Texans,...,-0.048,0.069,78,1019,2,0,Regular Season,2020,Houston Texans,-4.1
353,2020,2,TB,9,Sun,20-Sep,1:00:00 PM,1,Tampa Bay Buccaneers,Carolina Panthers,...,0.031,0.357,48,766,6,0,Regular Season,2020,Carolina Panthers,-0.9
354,2020,6,TB,9,Sun,18-Oct,4:25:00 PM,1,Tampa Bay Buccaneers,Green Bay Packers,...,0.031,0.357,48,525,2,0,Regular Season,2020,Green Bay Packers,-1.4


In [589]:
new_Df = new_Df[new_Df.Year == 2020]

In [590]:
new_Df.shape

(30, 76)

In [541]:
new_Df=new_Df.loc[:, ["Year", 'TEAM_POINTS', 'Power Index FPI', 'Power Index OFF',  'Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP', 'TEAM_TURNOVERS', 'TEAM_YARDS', 'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner','DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [620]:
missing_ones2 = missing_ones.loc[:, ["GAMEID", "FINAL_TEAM_NAME", "Model1_Prediction",
                                     "Model2_Prediction",  "Model3_Prediction",
                                     "Model4_Prediction", "Model5_Prediction"]]

In [622]:
missing_ones2

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction,Model2_Prediction,Model3_Prediction,Model4_Prediction,Model5_Prediction
0,5657,Tennessee Titans,23.725884,20.559308,20.154583,24.0,26.024176
1,5678,Tennessee Titans,23.725884,27.802101,28.089752,27.0,28.128450
2,5577,Tampa Bay Buccaneers,26.471791,24.286553,21.315701,20.0,19.747656
3,5696,Tampa Bay Buccaneers,26.471791,24.286553,27.268988,24.0,7.474485
4,5628,Tampa Bay Buccaneers,26.471791,20.363373,16.529400,20.0,20.423419
5,5690,Tennessee Titans,23.725884,21.162874,17.175795,10.0,17.833577
6,5690,Chicago Bears,21.426985,20.831442,16.703918,17.0,14.266240
7,5580,Denver Broncos,21.043835,20.436777,26.102005,27.0,19.257512
8,5586,Jacksonville Jaguars,17.148478,20.564305,24.180355,27.0,29.469500
9,5601,Minnesota Vikings,23.981317,26.282363,30.258027,31.0,26.127306


In [623]:
final_dd = pd.concat([model_predictions_fin, missing_ones2], axis=0)

In [629]:
final_dd

,GAMEID,FINAL_TEAM_NAME,Model1_Prediction,Model2_Prediction,Model3_Prediction,Model4_Prediction,Model5_Prediction
0,1.0,Detroit Lions,21.490843,25.956638,26.019291,27.0,27.131508
1,1.0,Chicago Bears,21.107693,24.249333,19.114992,20.0,22.373294
2,2.0,Chicago Bears,21.107693,24.249333,29.161770,27.0,26.021331
3,2.0,Detroit Lions,21.490843,25.956638,33.239537,24.0,31.618060
4,3.0,Washington Redskins,20.979977,25.486695,26.261249,24.0,30.166242
...,...,...,...,...,...,...,...
25,5641.0,Tennessee Titans,23.725884,25.086053,25.164547,27.0,33.156365
26,5650.0,Tennessee Titans,23.725884,29.009233,40.267880,27.0,41.811288
27,5587.0,Tampa Bay Buccaneers,26.471791,24.990713,22.249573,20.0,25.176297
28,5652.0,Tampa Bay Buccaneers,26.471791,25.493685,25.088920,20.0,29.897627


In [635]:
final_df = pd.read_csv("perfectlyCleanedData4.csv")

In [636]:
final_df = pd.merge(final_df, final_dd, left_on=["FINAL_TEAM_NAME", "GAMEID"], right_on=["FINAL_TEAM_NAME", "GAMEID"])

In [638]:
final_df.to_csv("webData.csv", index=False)